<a href="https://colab.research.google.com/github/francis008/fake-news-detection/blob/main/spam_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

# Reading the data
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/news.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/news.csv'

In [ ]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1,  padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [ ]:
embeddings_index = {}
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                              input_length=max_length, weights=[
                                  embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 54, 50)            377600    
                                                                 
 dropout_2 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                      

In [ ]:
num_epochs = 20

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded,
                                     testing_labels),
                    verbose=2)

Train on 2700 samples, validate on 300 samples
Epoch 1/20
2700/2700 - 3s - loss: 0.6576 - accuracy: 0.5985 - val_loss: 0.6205 - val_accuracy: 0.6733 - 3s/epoch - 1ms/sample
Epoch 2/20
2700/2700 - 2s - loss: 0.5680 - accuracy: 0.7100 - val_loss: 0.5421 - val_accuracy: 0.7000 - 2s/epoch - 726us/sample
Epoch 3/20
2700/2700 - 3s - loss: 0.5085 - accuracy: 0.7504 - val_loss: 0.5122 - val_accuracy: 0.7000 - 3s/epoch - 1ms/sample
Epoch 4/20
2700/2700 - 3s - loss: 0.4416 - accuracy: 0.7981 - val_loss: 0.5543 - val_accuracy: 0.7133 - 3s/epoch - 951us/sample
Epoch 5/20
2700/2700 - 2s - loss: 0.3780 - accuracy: 0.8311 - val_loss: 0.4909 - val_accuracy: 0.7533 - 2s/epoch - 720us/sample
Epoch 6/20
2700/2700 - 2s - loss: 0.2901 - accuracy: 0.8807 - val_loss: 0.5214 - val_accuracy: 0.7767 - 2s/epoch - 715us/sample
Epoch 7/20
2700/2700 - 2s - loss: 0.2451 - accuracy: 0.9033 - val_loss: 0.6116 - val_accuracy: 0.7500 - 2s/epoch - 724us/sample
Epoch 8/20
2700/2700 - 2s - loss: 0.2155 - accuracy: 0.9163 -

In [ ]:
# sample text to check if fake or not
X = "COVID-19 is fake"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type,
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")

NameError: name 'tokenizer1' is not defined